In [13]:
import requests
import pandas as pd
from datetime import datetime
import os

In [4]:
rest_api = 'https://ftx.com/api'
markets_endpoint = '/markets'
historical_price_endpoint = '/markets/{}/candles?resolution={}&start_time={}&end_time={}'

In [5]:
UNIX_START = pd.Timestamp(0, unit='s', tz='UTC')
MAX_RESOLUTION = 86400 * 30
API_RECORD_LIMIT = 1500

In [6]:
def get_unix_ts(date_str):
    return (pd.Timestamp(date_str, tz='UTC') - UNIX_START) // pd.Timedelta('1s')

def current_time():
    return get_unix_ts(datetime.utcnow())

In [7]:
def get_market_start(market_name):
    url = rest_api + historical_price_endpoint.format(market_name, MAX_RESOLUTION, 0, current_time())

    res = requests.get(url)
    if res.status_code != 200:
        raise f"Status Code is {res.status_code}, Something went wrong"

    prices = res.json()['result']
    if not len(prices):
        raise f"No price data was found"

    unix_ts = int(prices[0]['time'] // 1e3)
    return unix_ts


In [8]:
def get_all_market_prices(market_name, resolution):

    market_start = get_market_start(market_name)
    market_end = current_time() 

    df_list = []
    for start_ts in range(market_start, market_end, resolution * API_RECORD_LIMIT):
        end_ts = start_ts + resolution * (API_RECORD_LIMIT - 1)
        url = rest_api + historical_price_endpoint.format(market_name, resolution, start_ts, end_ts)
        res = requests.get(url)
        df_list.append(res.json()['result'])

    flattened_df_list = [item for sublist in df_list for item in sublist]
    df = pd.DataFrame(flattened_df_list)
    return df

In [9]:
res = requests.get(rest_api + markets_endpoint)
markets = pd.DataFrame(res.json()['result'])
top_10_markets = markets.iloc[markets.volumeUsd24h.nlargest(10).index].name.values

In [19]:
resolution = 900

for market in top_10_markets:
    print(f"Fetching Data for the {market} Market")
    df = get_all_market_prices(market, resolution)

    # ensure correct folders exist
    folder = '/'.join(f'data/{market}.csv'.split('/')[:-1])
    if not os.path.exists(folder):
        os.makedirs(folder)

    df.to_csv(f'data/{market}.csv')

Fetching Data for the ETH-PERP Market
Fetching Data for the BTC-PERP Market
Fetching Data for the ETH/USD Market
Fetching Data for the DOGE-PERP Market
Fetching Data for the BTC/USD Market
Fetching Data for the SOL-PERP Market
Fetching Data for the ETC-PERP Market
Fetching Data for the SOL/USD Market
Fetching Data for the MATIC-PERP Market
Fetching Data for the AVAX-PERP Market
